In [ ]:
input_path = "s3a://mkt-insight-datahub/mail/cdis.csv"
send_from = "aiweaver@emart.com"
send_to = "aiweaver@emart.com"
mail_subject = "Default subject"
mail_content = "Default content"

In [ ]:
from pydatafabric.ye import get_spark


spark = get_spark()

df = spark.read.csv(input_path)
df.count()

In [ ]:
from pydatafabric.vault_utils import get_secrets
from pydatafabric.github_utils import GithubUtil


secrets = get_secrets('github/sktaiflow')
token = secrets['token']
proxies = {
    'http': secrets['proxy'],
    'https': secrets['proxy'],
}
g = GithubUtil(token, proxies=proxies)
fuction_code = g.download_from_git(f'https://github.com/sktaiflow/crypto-function/blob/master/module/crypto.py')
exec(fuction_code)

In [ ]:
result = datafabric_crypto(ss=spark, df=df, before="lake_hash", after="raw", key_column="_c0").collect()
result_list = [row["_c0"] for row in result]
len(result_list)

In [ ]:
def send_email(subject, text, send_from, send_to, attachment, filename):
    """
    :param str attachment: Attachment to send as .txt file with email
    """
    import smtplib
    from email.mime.application import MIMEApplication
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.utils import formatdate
    from skt.vault_utils import get_secrets

    c = get_secrets(path="mail")
    host, port = c["smtp_host"], c["smtp_port"]
    msg = MIMEMultipart()
    msg["From"] = send_from
    msg["To"] = send_to
    msg["Date"] = formatdate(localtime=True)
    msg["Subject"] = subject
    msg.attach(MIMEText(text))

    if attachment:
        part = MIMEApplication(attachment, NAME=filename)
        part.add_header("Content-Disposition", f"attachment; filename={filename}")
        msg.attach(part)

    with smtplib.SMTP(host, port) as smtp:
        return smtp.sendmail(send_from, send_to.split(","), msg.as_string())

att = "\n".join(result_list)
filename = input_path.split("/")[-1]
send_email(mail_subject, mail_content, send_from, send_to, attachment=att, filename=filename)

In [ ]:
spark.stop()